In [2]:
from plyfile import PlyData
import jax 
import jax.numpy as jnp 
from jax import grad, jacfwd, vmap, jit
import polyscope as ps

from jaxlie import SO3, SE3
from jax.flatten_util import ravel_pytree

import numpy as np
import open3d as o3d 
import trimesh as tm

import meshcat
from meshcat import Visualizer
import meshcat.geometry as mc_geom
import meshcat.transformations as mc_trans

from kernel import create_kernel_matrix, RBF
import jaxopt

import time

objc[104]: Class GLFWWindowDelegate is implemented in both /Users/ia285/miniconda3/lib/python3.10/site-packages/polyscope_bindings.cpython-310-darwin.so (0x11eaeae90) and /Users/ia285/miniconda3/lib/python3.10/site-packages/open3d/cpu/pybind.cpython-310-darwin.so (0x17b142db0). One of the two will be used. Which one is undefined.
objc[104]: Class GLFWApplicationDelegate is implemented in both /Users/ia285/miniconda3/lib/python3.10/site-packages/polyscope_bindings.cpython-310-darwin.so (0x11eaeae68) and /Users/ia285/miniconda3/lib/python3.10/site-packages/open3d/cpu/pybind.cpython-310-darwin.so (0x17b142e28). One of the two will be used. Which one is undefined.
objc[104]: Class GLFWContentView is implemented in both /Users/ia285/miniconda3/lib/python3.10/site-packages/polyscope_bindings.cpython-310-darwin.so (0x11eaeaee0) and /Users/ia285/miniconda3/lib/python3.10/site-packages/open3d/cpu/pybind.cpython-310-darwin.so (0x17b142e50). One of the two will be used. Which one is undefined.
ob

In [10]:
norm = np.array([1., 1., 1.])
norm = norm/np.linalg.norm(norm)
root = np.array([1., 0., 0.])

args = {'norm' : norm, 'root' : root}
v1 = np.array([0., 0., 1.])
def loss(params, args):
    w = params['w']
    v = params['v']
    norm = args['norm']
    root = args['root']
    _R = SO3.exp(w)
    return jnp.sum((norm+_R@v1)**2) + np.sum((norm+root-v)**2)

In [4]:
vis = Visualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [5]:
body1 = vis['v1']
body1.set_object(mc_geom.triad(scale=0.5))

body2 = vis['v2']
body2.set_object(mc_geom.triad(scale=0.5))


_angle = np.arccos(np.dot(np.array([0.,0.,1.]), norm))
_axis = np.cross(np.array([0.,0.,1.]), norm)
_R = SO3.exp(_axis*_angle)
_T = SE3.from_rotation_and_translation(_R, root).as_matrix()
body2.set_transform(np.array(_T, dtype=np.float64))



I0000 00:00:1723150872.261561       1 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


In [44]:
w = jnp.array([0.1,0.2,0.3])
v = jnp.array([.0,0.,.01])

params = {'w': w, 'v':v}
for _ in range(100):
    _grad = grad(loss)(params, args)
    params['w'] = params['w'] - 0.1*_grad['w']
    params['v'] = params['v'] - 0.1*_grad['v']
    _R = SO3.exp(params['w'])
    _T = SE3.from_rotation_and_translation(_R, params['v']).as_matrix()
    body1.set_transform(np.array(_T, dtype=np.float64))
    # print(params, loss(params, args))
    time.sleep(0.1)

KeyError: 'norm'

In [4]:
dataset = o3d.data.BunnyMesh()
plydata = PlyData.read(dataset.path)
vertices = np.vstack((
    plydata['vertex']['x'],
    plydata['vertex']['y'],
    plydata['vertex']['z']
)).T

# _min_pnt = vertices.min(axis=0)
# _max_pnt = vertices.max(axis=0)
# _mid_pnt = vertices.mean(axis=0)
# vertices = (vertices - _mid_pnt)/(_max_pnt-_min_pnt)

faces = np.array(np.vstack(plydata['face']['vertex_indices']))

mesh = tm.Trimesh(vertices=vertices, faces=faces)

ds_mesh = mesh.simplify_quadric_decimation(int(mesh.vertices.shape[0]/10))

In [ ]:
# ps.init()

# ps_mesh = ps.register_surface_mesh("my mesh", ds_mesh.vertices, ds_mesh.faces)
# ps_mesh.add_vector_quantity("rand vecs", ds_mesh.vertex_normals, enabled=True)


# ps.show()

In [48]:
XP = jnp.hstack([ds_mesh.vertex_normals, ds_mesh.vertices])
X  = jnp.zeros((100,6))

In [90]:
X_samples = ds_mesh.vertices+ds_mesh.vertex_normals*0.05
P_XI      = jnp.ones(X_samples.shape[0])
P_XI = P_XI/np.sum(P_XI)
h = 0.01
args = {'h' : h, 'vertices' : ds_mesh.vertices, 'normals' : ds_mesh.vertex_normals}

In [1]:
v1 = np.array([0., 0., 1.])
def phi(x, xp, h=0.01):
    w, v        = jnp.split(x,2)
    norm, root  = jnp.split(xp, 2)
    _R = SO3.exp(w)
    return jnp.exp(
        -(jnp.sum((norm+_R@v1)**2)+ np.sum((norm+root-v)**2))/h
    )

def RBF(x, xp, h=0.01):
    w, v        = jnp.split(x,2)
    norm, root  = jnp.split(xp, 2)
    return np.exp(
        -np.sum((v-root)**2 + (norm-w)**2)/h
    )

NameError: name 'np' is not defined

In [92]:
phi_vmapped = vmap(vmap(phi, in_axes=(0,None,None)),in_axes=(None,0,None))

In [95]:
KernelMatrix = create_kernel_matrix(RBF)

x0 =  X_samples.min(axis=0)
xf =  X_samples.max(axis=0)
T = 80
X_init = np.linspace(x0, xf, num=T, endpoint=True)
V_init = np.zeros_like(X_init)

X = jnp.hstack([V_init, X_init])

sol, unflatten_X = ravel_pytree(X)
bounds = (-np.inf * np.ones_like(sol), np.inf * np.ones_like(sol))

def ergodic_mmd(flat_X, args):
    X_init = unflatten_X(flat_X)
    w, v        = jnp.split(X_init,2, axis=1)

    T = X_init.shape[0]
    h = args['h']
    # x0 = args['x0']
    vertices = args['vertices']
    norms    = args['normals']
    XP = jnp.hstack([norms, vertices])
    # P_XI      = jnp.sum(phi_vmapped(X_init, XP, h), axis=1)
    return np.sum(KernelMatrix(v, v, h))/(T**2) \
            - 2 * np.sum(P_XI @ KernelMatrix(v, vertices+norms, h))/T \
            + np.mean((v[1:]-v[:-1])**2)
            # + np.sum((X_init[0]-x0)**2)

solver = jaxopt.ProjectedGradient(fun=ergodic_mmd, projection=jaxopt.projection.projection_box, tol=1e-6)
solver_state = solver.init_state(sol, hyperparams_proj=bounds)
# sol = solver.run(init_params=sol, 
#                         hyperparams_proj=bounds,  
#                         args=args).params

X = unflatten_X(sol)

In [96]:
ps.init()

ps_bunny_mesh = ps.register_surface_mesh("bunny", ds_mesh.vertices, ds_mesh.faces)
ps_bunny_mesh.set_transparency(0.8)
ps_traj         = ps.register_curve_network("trajectory", X_init * 1.25, edges="line")

for _ in range(1000):
    (sol, solver_state) = solver.update(
                        params=sol, 
                        state=solver_state, 
                        hyperparams_proj=bounds, 
                                args=args)
    X = unflatten_X(sol)
    ps_traj.update_node_positions(X[:,:3])
    ps.frame_tick()

    time.sleep(0.001)

KeyboardInterrupt: 

In [23]:
ps.init()

ps_bunny_mesh = ps.register_surface_mesh("bunny", ds_mesh.vertices, ds_mesh.faces)
ps_bunny_mesh.set_transparency(0.8)
# ps_bunny_mesh.add_scalar_quantity('info_distr', mesh_func.func_vals, defined_on='vertices', cmap='blues')

ps_traj         = ps.register_curve_network("trajectory", X * 1.25, edges="line")

ps.show()

KeyboardInterrupt: 

In [31]:
lie_RBK_Grammian(X, XP, 0.1).shape

(1766, 100)

In [22]:
def lie_RBF(x, xp, h=0.01):
    w, v        = jnp.split(x,2)
    norm, root  = jnp.split(xp, 2)
    _R = SO3.exp(w)
    return jnp.exp(
        -(jnp.sum((norm+_R@v1)**2) + np.sum((norm+root-v)**2))/h
    )

def create_kernel_matrix(kernel):
    return vmap(vmap(kernel, in_axes=(0, None, None)), in_axes=(None, 0, None))